Выполнила Леопаева Л.А., 6402-010302D

Лабораторная работа 1

In [2]:
import numpy as np
from itertools import combinations

1.1. Реализовать функцию REF(), приводящую матрицу к
ступенчатому виду.

In [7]:
def REF(mat):
    matrix = np.array(mat)
    rows, cols = matrix.shape
    leading_col = 0
    for r in range(rows):
        if leading_col >= cols:
            return matrix
        i = r
        while matrix[i, leading_col] == 0:
            i += 1
            if i == rows:
                i = r
                leading_col += 1
                if leading_col == cols:
                    return matrix
        matrix[[i, r]] = matrix[[r, i]]

        for i in range(r + 1, rows):
            if matrix[i, leading_col] != 0:
                matrix[i] = (matrix[i] + matrix[r]) % 2
        leading_col += 1

    return matrix


S = np.array([
        [1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
        [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
        [1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0],
        [1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0]])

S_REF = REF(S)
print(f"\nПорождающая матрица в ступенчатом виде на основе входной:\n{S_REF}")


Порождающая матрица в ступенчатом виде на основе входной:
[[1 0 1 1 0 0 0 1 0 0 1]
 [0 0 0 1 1 1 0 1 0 1 0]
 [0 0 0 0 1 0 0 1 0 0 1]
 [0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0]]


1.2. Реализовать функцию RREF(), приводящую матрицу к
приведённому ступенчатому виду. 

In [8]:
def RREF(inp_matrix):
    matrix = REF(inp_matrix)
    rows, cols = matrix.shape
    leading_col = 0

    for r in range(rows):
        # Пропускаем строки с нулями
        while leading_col < cols and matrix[r, leading_col] == 0:
            leading_col += 1
        if leading_col >= cols:
            break

        # Обнуляем элементы выше ведущего элемента
        for i in range(r):
            if matrix[i, leading_col] != 0:
                matrix[i] = (matrix[i] + matrix[r]) % 2

    non_zero_rows = matrix[~np.all(matrix == 0, axis=1)]
    return non_zero_rows


S_RREF = RREF(S)
print(f"\nСтупенчатая матрица по модулю 2: \n{S_RREF}")


Ступенчатая матрица по модулю 2: 
[[1 0 1 0 0 1 0 1 0 1 0]
 [0 0 0 1 0 1 0 0 0 1 1]
 [0 0 0 0 1 0 0 1 0 0 1]
 [0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 1 1]]


1.3. Создать класс линейных кодов LinearCode. 

In [19]:
class LinearCode:
    def __init__(self, generator_matrix):
        """
        Инициализация линейного кода.
        """
        self.G = np.array(generator_matrix)
        self.G = self.RREF(self.G)
        self.k, self.n = self.G.shape

    def REF(self, mat):
        """
        Приведение матрицы к ступенчатому виду.
        """
        matrix = np.array(mat)
        rows, cols = matrix.shape
        leading_col = 0
        for r in range(rows):
            if leading_col >= cols:
                return matrix
            i = r
            while matrix[i, leading_col] == 0:
                i += 1
                if i == rows:
                    i = r
                    leading_col += 1
                    if leading_col == cols:
                        return matrix
            matrix[[i, r]] = matrix[[r, i]]

            for i in range(r + 1, rows):
                if matrix[i, leading_col] != 0:
                    matrix[i] = (matrix[i] + matrix[r]) % 2
            leading_col += 1

        return matrix

    def RREF(self, inp_matrix):
        """
        Приведение матрицы к приведённому ступенчатому виду.
        """
        matrix = self.REF(inp_matrix)
        rows, cols = matrix.shape
        leading_col = 0

        for r in range(rows):
            # Пропускаем строки с нулями
            while leading_col < cols and matrix[r, leading_col] == 0:
                leading_col += 1
            if leading_col >= cols:
                break

            # Обнуляем элементы выше ведущего элемента
            for i in range(r):
                if matrix[i, leading_col] != 0:
                    matrix[i] = (matrix[i] + matrix[r]) % 2

        non_zero_rows = matrix[~np.all(matrix == 0, axis=1)]
        return non_zero_rows

    def find_leading_columns(self):
        """
        Находит индексы ведущих столбцов матрицы.
        """
        lead_columns = []
        for row in self.G:
            lead_index = np.where(row != 0)[0]
            if lead_index.size > 0:
                lead_columns.append(int(lead_index[0]))
        return lead_columns

    def reduce_matrix(self, lead_columns):
        """
        Формирует сокращённую матрицу, удаляя ведущие столбцы из матрицы.
        """
        mask = np.ones(self.n, dtype=bool)
        mask[lead_columns] = False
        reduced_matrix = self.G[:, mask]
        return reduced_matrix

    def compute_parity_check_matrix(self, lead_columns, reduced_matrix):
        """
        Формирует матрицу H, помещая строки из сокращённой матрицы X в строки,
        соответствующие ведущим столбцам, и заполняя остальные строки единичной матрицей.
        """
        identity_matrix = np.eye(self.n - len(lead_columns), dtype=int)

        result = []
        total_rows = len(lead_columns) + identity_matrix.shape[0]

        x_index = 0
        for i in range(total_rows):
            if x_index < len(reduced_matrix) and (x_index < len(lead_columns) and lead_columns[x_index] == i):
                result.append(reduced_matrix[x_index])
                x_index += 1
            else:
                result.append(identity_matrix[i - x_index])

        return np.array(result)


    def generate_codewords_1(self):
        '''
        Формирование кодовых слов длиной n
        путем сложения слов из порождающего множества.
        '''
        rows = self.G.shape[0]
        codewords = set()

        for r in range(1, rows + 1):
            for comb in combinations(range(rows), r):
                codeword = np.bitwise_xor.reduce(self.G[list(comb)], axis=0)
                codewords.add(tuple(codeword))

        codewords.add(tuple(np.zeros(self.n, dtype=int)))

        return np.array(list(codewords))


    def generate_codewords_2(self):
        '''
        Формирование кодовых слов длиной n
        путем умножения всех
        двоичных слов длины k на G.
        '''
        k = self.k
        codewords = []
        for i in range(2 ** k):
            binary_word = np.array(list(np.binary_repr(i, width=k)), dtype=int)
            codeword = np.dot(binary_word, self.G) % 2
            codewords.append(codeword)

        return np.array(codewords)


    def check_codeword(self, H, codeword):
        '''
        Функция проверки кодовых слов
        путем умножения на проверочную матрицу.
        '''
        codeword_reshaped = codeword.reshape(1, -1)
        return np.dot(codeword_reshaped, H) % 2

    def compute_distance(self):
        """
        Вычисляет кодовое расстояние d.
        """
        codewords = self.generate_codewords_2()
        non_zero_codewords = codewords[codewords.any(axis=1)]
        distances = np.sum(non_zero_codewords, axis=1)

        d = np.min(distances) if distances.size > 0 else 0

        return d

    def compute_t(self, distance):
        """
        Вычисляет кратность обнаруживаемой ошибки t.
        """
        # t = (d - 1) // 2
        t = (d - 1)
        return t

    def make_error_in_code(self, codeword, error_count):
        """
        Вносит ошибки в кодовое слово в указанные позиции.
        """
        error_vector = np.zeros_like(codeword)
        error_positions = np.random.choice(len(codeword), error_count, replace=False)
        for pos in error_positions:
            error_vector[pos] = 1

        return codeword + error_vector % 2, error_vector

    def is_error_detected(self, H, codeword_with_error):
        """
        Проверяет, обнаруживается ли ошибка в кодовом слове.
        """
        return np.any(self.check_codeword(H, codeword_with_error) != 0)

    def __repr__(self):
        return f"LinearCode(G=\n{self.G})"


1.3.2 Задать n равное числу столбцов и k равное числу строк
полученной матрицы (без учёта полностью нулевых строк).


In [20]:
G = np.array([
        [1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
        [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]])

print('G =\n', G)

n = G.shape[1]
# Количество ненулевых строк k
k = np.sum(np.any(G != 0, axis=1))
print(f"n = {n},\nk = {k}")

G =
 [[1 0 1 1 0 0 0 1 0 0 1]
 [0 0 0 1 1 1 0 1 0 1 0]
 [0 0 0 0 1 0 0 1 0 0 1]
 [0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 1 1]]
n = 11,
k = 5


1.3.3 Сформировать проверочную матрицу на основе порождающей. 

In [21]:
G_lc = LinearCode(G)
print('G* =\n', G_lc.G)

lead = G_lc.find_leading_columns()
print(f"\nВедущие столбцы: {lead}")

X = G_lc.reduce_matrix(lead)
print('\nСокращённая матрица X =\n', X)

H = G_lc.compute_parity_check_matrix(lead, X)
print('\nМатрица H =\n', H)

G* =
 [[1 0 1 0 0 1 0 1 0 1 0]
 [0 0 0 1 0 1 0 0 0 1 1]
 [0 0 0 0 1 0 0 1 0 0 1]
 [0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 1 1]]

Ведущие столбцы: [0, 3, 4, 6, 8]

Сокращённая матрица X =
 [[0 1 1 1 1 0]
 [0 0 1 0 1 1]
 [0 0 0 1 0 1]
 [0 0 0 0 1 0]
 [0 0 0 0 1 1]]

Матрица H =
 [[0 1 1 1 1 0]
 [1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 1 1]
 [0 0 0 1 0 1]
 [0 0 1 0 0 0]
 [0 0 0 0 1 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 1]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]]


1.4. Сформировать все кодовые слова длины n двумя способами.


In [24]:
# Генерация кодовых слов
codewords_1 = G_lc.generate_codewords_1()
print("Кодовые слова из порождающего множества:")
print(codewords_1)

codewords_2 = G_lc.generate_codewords_2()
print("Кодовые слова из двоичных слов:")
print(codewords_2)

# Проверка на совпадение
assert set(map(tuple, codewords_1)) == set(map(tuple, codewords_2)), "Кодовые слова не совпадают"

# Проверяем, что кодовые слова при умножении на H дают нулевые вектора
for codeword in codewords_1:
    assert np.all(G_lc.check_codeword(H, codeword) == 0), "Кодовое слово не дает нулевой вектор при умножении на H"

print("\nВсе проверки пройдены успешно!")

Кодовые слова из порождающего множества:
[[0 0 0 0 1 0 1 1 1 0 0]
 [1 0 1 0 1 1 1 0 1 1 0]
 [1 0 1 0 0 1 1 1 0 0 0]
 [1 0 1 1 0 0 0 1 0 0 1]
 [1 0 1 1 1 0 1 0 0 1 0]
 [0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 1 0 1 1 0 1 1 0]
 [0 0 0 0 1 0 1 1 0 1 1]
 [0 0 0 1 0 1 0 0 0 1 1]
 [0 0 0 1 0 1 0 0 1 0 0]
 [1 0 1 0 1 1 0 0 1 0 0]
 [0 0 0 0 1 0 0 1 1 1 0]
 [0 0 0 0 1 0 0 1 0 0 1]
 [0 0 0 0 0 0 1 0 1 0 1]
 [1 0 1 0 1 1 1 0 0 0 1]
 [1 0 1 1 0 0 1 1 1 0 0]
 [1 0 1 1 0 0 0 1 1 1 0]
 [1 0 1 1 1 0 0 0 1 1 1]
 [0 0 0 1 1 1 0 1 1 0 1]
 [0 0 0 0 0 0 0 0 1 1 1]
 [1 0 1 0 0 1 0 1 0 1 0]
 [1 0 1 0 1 1 0 0 0 1 1]
 [1 0 1 1 1 0 0 0 0 0 0]
 [1 0 1 1 0 0 1 1 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 1 1 0 0 0 1]
 [1 0 1 0 0 1 0 1 1 0 1]
 [1 0 1 0 0 1 1 1 1 1 1]
 [0 0 0 1 1 1 0 1 0 1 0]
 [0 0 0 1 1 1 1 1 1 1 1]
 [0 0 0 1 1 1 1 1 0 0 0]]
Кодовые слова из двоичных слов:
[[0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1]
 [0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 1 0 1 0 1]
 [0 0 0 0 1 0 0 1 0 0 1]
 

1.4 Вычислить кодовое расстояние получившегося кода. 

In [27]:
d = G_lc.compute_distance()
t = G_lc.compute_t(d)
print(f"Кодовое расстояние d = {d}\nКратность обнаруживаемой ошибки t = {t}")

Кодовое расстояние d = 2
Кратность обнаруживаемой ошибки t = 1


1.4.1 Внести в кодовое слово ошибку кратности не более t, умножить
полученное слово на H, убедиться в обнаружении ошибки. 

In [29]:
print("Ошибка кратности не более t")
v = codewords_1[0]
print(f"\nv = {v}")
e1_codeword, e1_vector = G_lc.make_error_in_code(v, t)
print(f"e1 = {e1_vector}")
print(f"v + e1 = {e1_codeword}")
print(f"(v + e1) @ H = {G_lc.check_codeword(H, e1_codeword)} – error")

Ошибка кратности не более t

v = [0 0 0 0 1 0 1 1 1 0 0]
e1 = [0 0 1 0 0 0 0 0 0 0 0]
v + e1 = [0 0 1 0 1 0 1 1 1 0 0]
(v + e1) @ H = [[0 1 0 0 0 0]] – error


1.4.2 Найти для некоторого кодового слова ошибку кратности t+1
такую, что при умножении на H ошибка не может быть обнаружена.

In [30]:
print("Ошибка кратности t+1")
v = codewords_1[1]
print(f"\nv = {v}")
e2_codeword, e2_vector = G_lc.make_error_in_code(v, t + 1)
print(f"e2 = {e2_vector}")
print(f"v + e2 = {e2_codeword}")
print(f"(v + e2) @ H = {G_lc.check_codeword(H, e2_codeword)} – no error")

Ошибка кратности t+1

v = [1 0 1 0 1 1 1 0 1 1 0]
e2 = [0 0 0 1 0 0 0 1 0 0 0]
v + e2 = [1 0 1 1 1 1 1 1 1 1 0]
(v + e2) @ H = [[0 0 1 1 1 1]] – no error
